In [634]:
print('Guck Guck')

Guck Guck


In [635]:
import numpy as np
import pandas as pd
import os  

In [719]:
import chardet

# look at the first 100k bytes to guess the character encoding
with open("Sample-Superstore.csv", 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))

# check what the character encoding might be
print(result)

{'encoding': 'Windows-1252', 'confidence': 0.73, 'language': ''}


In [720]:
superstore = pd.read_csv('Sample-Superstore.csv', encoding='Windows-1252')

In [721]:
superstore.head(2)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,3,0.0,219.5820


In [722]:
superstore.shape

(9994, 21)

In [723]:
superstore.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9994 non-null   int64  
 1   Order ID       9994 non-null   object 
 2   Order Date     9994 non-null   object 
 3   Ship Date      9994 non-null   object 
 4   Ship Mode      9994 non-null   object 
 5   Customer ID    9994 non-null   object 
 6   Customer Name  9994 non-null   object 
 7   Segment        9994 non-null   object 
 8   Country        9994 non-null   object 
 9   City           9994 non-null   object 
 10  State          9994 non-null   object 
 11  Postal Code    9994 non-null   int64  
 12  Region         9994 non-null   object 
 13  Product ID     9994 non-null   object 
 14  Category       9994 non-null   object 
 15  Sub-Category   9994 non-null   object 
 16  Product Name   9994 non-null   object 
 17  Sales          9994 non-null   float64
 18  Quantity

In [724]:
#Path to CSV save location - do not save any other CSVs here which will not be tables in the database!
path = r"C:\Users\gabb_\OneDrive\Desktop\SQLite Challange\Superstore_Project\tables"
os.makedirs(path, exist_ok=True)

In [725]:
#We need to fix OrderDate and ship date to be YYYY/MM/DD (can be ordered easily even when seen as a string). 
superstore[['Order Date','Ship Date']] = superstore[['Order Date','Ship Date']].apply(pd.to_datetime)

In [726]:
superstore.head(1)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136


In [727]:
superstore['Profit'].round(2)

0        41.91
1       219.58
2         6.87
3      -383.03
4         2.52
         ...  
9989      4.10
9990     15.63
9991     19.39
9992     13.32
9993     72.95
Name: Profit, Length: 9994, dtype: float64

In [728]:
superstore['Sales'].round(2)

0       261.96
1       731.94
2        14.62
3       957.58
4        22.37
         ...  
9989     25.25
9990     91.96
9991    258.58
9992     29.60
9993    243.16
Name: Sales, Length: 9994, dtype: float64

In [729]:
# Two new columns required for more in depth analysis. 
# Supplier price (what VS Group buy it for) and Catalogue price (what VS Group offer it for)

superstore["ProductPrice"] = ((superstore.Sales/superstore.Quantity)/(1-superstore.Discount)).round(2)

superstore["SupplierPrice"] = ((superstore.Sales-superstore.Profit)/superstore.Quantity).round(2)


In [730]:
#Sales, Profit and Supplier Price need to be rounded to the nearest cent. 
#All prices will also need to be in CENTS to avoid floating point errors in SQLite.

cols = ["Sales", "Profit", "SupplierPrice", "ProductPrice"]

superstore.loc[:, cols] = (superstore[cols]*100).astype(int)

In [731]:

superstore.head(5)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,ProductPrice,SupplierPrice
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,26195.0,2,0.00,4191.0,13097.0,11002.0
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",73194.0,3,0.00,21958.0,24398.0,17079.0
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,1462.0,2,0.00,687.0,731.0,387.0
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,95757.0,5,0.45,-38303.0,34821.0,26812.0
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,2236.0,2,0.20,251.0,1398.0,993.0


In [732]:
# we have several products with more than one prices, we need to investigate that later but


#Let's get these column names into something more SQLitey 
rename_dict = {
        'Row ID': 'RowID',
        'Order ID': 'OrderID',
        'Order Date': 'OrderDate',
        'Ship Date': 'ShipDate',
        'Ship Mode': 'ShipMode',
        'Customer ID': 'CustomerID',
        'Customer Name': 'CustomerName',
        'Postal Code': 'PostalCode',
        'Product ID': 'ProductID',
        'Sub-Category': 'SubCategory',
        'Product Name': 'ProductName',
        'Segment':'CustomerType'
        }
 
# call rename () method
superstore.rename(columns=rename_dict,
          inplace=True)

In [733]:
superstore.columns

Index(['RowID', 'OrderID', 'OrderDate', 'ShipDate', 'ShipMode', 'CustomerID',
       'CustomerName', 'CustomerType', 'Country', 'City', 'State',
       'PostalCode', 'Region', 'ProductID', 'Category', 'SubCategory',
       'ProductName', 'Sales', 'Quantity', 'Discount', 'Profit',
       'ProductPrice', 'SupplierPrice'],
      dtype='object')

In [734]:
#Just trying some breakdowns to get a feel for the dataset.
#We can see that "Profit" is really just based on the catalogue price, we can still make money on "negative" sales.

#superstore.groupby(['State'])[["Sales", "Profit", "ProductPrice", "SupplierPrice"]].sum().sort_values(by=['Sales'], ascending=False)

# Structuring different tables for our database

In [735]:
# First pass

Customers = superstore[['CustomerID', 'CustomerName', 'CustomerType']]
Address = superstore[['CustomerID', 'Region', 'State', 'City', 'PostalCode']] # Address ID will be required to connect this to orders. Removed country as not required
Products = superstore[['ProductID', 'ProductName', 'ProductPrice', 'SupplierPrice', 'Category', 'SubCategory']] 
OrderDetails = superstore[['OrderID', 'ProductID', 'Quantity', 'Discount', 'OrderDate']] #Needs SalesID
Shipment=superstore[['ShipMode', 'ShipDate']]# needs SalesID and AddressID

# Customers

In [736]:
Customers.isnull().sum()

CustomerID      0
CustomerName    0
CustomerType    0
dtype: int64

In [737]:
Customers.head(3)

,CustomerID,CustomerName,CustomerType
0,CG-12520,Claire Gute,Consumer
1,CG-12520,Claire Gute,Consumer
2,DV-13045,Darrin Van Huff,Corporate


In [738]:
Customers["CustomerID"].unique().size

793

In [739]:
Customers["CustomerName"].unique().size

793

In [740]:
# checking if the customer types are consistent

# First, group by Customer ID 
customer_type_per_customer = superstore.groupby('CustomerID')['CustomerType'].nunique()

# Check if there are any Customer IDs with more than one customer type
conflicting_customer_type = customer_type_per_customer[customer_type_per_customer > 1]

if conflicting_customer_type.empty:
    print("All customer have consistent customer type.")
else:
    print("Customers with conflicting customer type:")
    print(conflicting_customer_type)

All customer have consistent customer type.


In [741]:
Customers = Customers.drop_duplicates(ignore_index=True)

In [742]:
## Customer table is ready
Customers.head(3)

,CustomerID,CustomerName,CustomerType
0,CG-12520,Claire Gute,Consumer
1,DV-13045,Darrin Van Huff,Corporate
2,SO-20335,Sean O'Donnell,Consumer


# Address

In [743]:
# prepre the address table
Address.isnull().sum()

CustomerID    0
Region        0
State         0
City          0
PostalCode    0
dtype: int64

In [744]:
Address.head(3)

,CustomerID,Region,State,City,PostalCode
0,CG-12520,South,Kentucky,Henderson,42420
1,CG-12520,South,Kentucky,Henderson,42420
2,DV-13045,West,California,Los Angeles,90036


In [745]:
# checking if the addreses are consistent per customerid

# First, group by Customer ID and get unique address
customerid_per_postalcode = superstore.groupby('CustomerID')['PostalCode'].nunique()

# Check if there are any Customer IDs with more than one unique segment
conflicting_postalcode = customerid_per_postalcode[customerid_per_postalcode > 1]

if conflicting_postalcode.empty:
    print("All customerid have consistent postalcode.")
else:
    print("Customerids with conflicting postalcode:")
    print(conflicting_postalcode)

Customerids with conflicting postalcode:
CustomerID
AA-10315     5
AA-10375     9
AA-10480     4
AA-10645     6
AB-10015     3
            ..
XP-21865    11
YC-21895     5
YS-21880     8
ZC-21910    13
ZD-21925     5
Name: PostalCode, Length: 780, dtype: int64


In [746]:
# this is clearly showing we have customers with more than one address(postalcode) which would mean that we need to create a unique addressid

In [747]:
Address=Address.drop_duplicates(ignore_index=True)

In [748]:
#We need to make a new column to use as a primary key for Addresses. That, or use a composite primary.. 
# Good discussion here https://stackoverflow.com/questions/14588304/composite-primary-key-vs-additional-id-column
#So we will use these as a reference to increase the number and we will put UNIQUE constraints on the composite, but use the ID as primary key.
Address['AddressID'] = range(1, len(Address) + 1)

In [749]:
Address['AddressID'] = "A" + Address['AddressID'].astype(str).str.zfill(6)

In [750]:
# table Address is ready
Address.tail(3)

,CustomerID,Region,State,City,PostalCode,AddressID
4907,TB-21400,South,Florida,Miami,33180,A004908
4908,DB-13060,West,California,Costa Mesa,92627,A004909
4909,CC-12220,West,California,Westminster,92683,A004910


# OrderDetails

In [751]:
# we would like to create a SalesID 

In [752]:
superstore['SalesID'] = range(1, len(superstore) + 1)

In [753]:
superstore['SalesID'] = "S" + superstore['SalesID'].astype(str).str.zfill(10)

In [754]:
superstore.tail(3)

,RowID,OrderID,OrderDate,ShipDate,ShipMode,CustomerID,CustomerName,CustomerType,Country,City,...,Category,SubCategory,ProductName,Sales,Quantity,Discount,Profit,ProductPrice,SupplierPrice,SalesID
9991,9992,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,Technology,Phones,Aastra 57i VoIP phone,25857.0,2,0.2,1939.0,16161.0,11959.0,S0000009992
9992,9993,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,Office Supplies,Paper,"It's Hot Message Books with Stickers, 2 3/4"" x 5""",2960.0,4,0.0,1332.0,740.0,407.0,S0000009993
9993,9994,CA-2017-119914,2017-05-04,2017-05-09,Second Class,CC-12220,Chris Cortes,Consumer,United States,Westminster,...,Office Supplies,Appliances,"Acco 7-Outlet Masterpiece Power Center, Wihtou...",24316.0,2,0.0,7294.0,12158.0,8511.0,S0000009994


In [755]:
OrderDetails = superstore[['SalesID', 'OrderID', 'ProductID', 'Quantity', 'Discount', 'OrderDate']]

In [756]:
# preparing orders table
OrderDetails.isnull().sum()

SalesID      0
OrderID      0
ProductID    0
Quantity     0
Discount     0
OrderDate    0
dtype: int64

In [757]:
OrderDetails.shape

(9994, 6)

# Shipment

In [758]:
# we need to add the addressid and the salesid

In [759]:
superstore_plus_addressid= Address.merge(superstore, left_on=['CustomerID', 'Region', 'State', 'City', 'PostalCode'], 
                     right_on=['CustomerID', 'Region', 'State', 'City', 'PostalCode'], 
                     how='inner').sort_values(by="RowID", ignore_index=True)

In [760]:
superstore_plus_addressid.columns

Index(['CustomerID', 'Region', 'State', 'City', 'PostalCode', 'AddressID',
       'RowID', 'OrderID', 'OrderDate', 'ShipDate', 'ShipMode', 'CustomerName',
       'CustomerType', 'Country', 'ProductID', 'Category', 'SubCategory',
       'ProductName', 'Sales', 'Quantity', 'Discount', 'Profit',
       'ProductPrice', 'SupplierPrice', 'SalesID'],
      dtype='object')

In [761]:
Shipment = superstore_plus_addressid[['SalesID', 'AddressID', 'ShipMode', 'ShipDate']]

In [762]:
Shipment.head(4)

,SalesID,AddressID,ShipMode,ShipDate
0,S0000000001,A000001,Second Class,2016-11-11
1,S0000000002,A000001,Second Class,2016-11-11
2,S0000000003,A000002,Second Class,2016-06-16
3,S0000000004,A000003,Standard Class,2015-10-18


In [763]:
Shipment.isnull().sum()

SalesID      0
AddressID    0
ShipMode     0
ShipDate     0
dtype: int64

In [764]:
Shipment.shape

(9994, 4)

# Products

In [765]:
Products.columns

Index(['ProductID', 'ProductName', 'ProductPrice', 'SupplierPrice', 'Category',
       'SubCategory'],
      dtype='object')

In [766]:
# preparing products table
Products.isnull().sum()

ProductID        0
ProductName      0
ProductPrice     0
SupplierPrice    0
Category         0
SubCategory      0
dtype: int64

In [767]:
Products=Products.drop_duplicates(ignore_index=True)

In [768]:
Products.sort_values('ProductID')

,ProductID,ProductName,ProductPrice,SupplierPrice,Category,SubCategory
1571,FUR-BO-10000112,"Bush Birmingham Collection Bookcase, Dark Cherry",13097.0,10478.0,Furniture,Bookcases
1132,FUR-BO-10000330,"Sauder Camden County Barrister Bookcase, Plank...",12098.0,10404.0,Furniture,Bookcases
1068,FUR-BO-10000362,Sauder Inglewood Library Bookcases,17098.0,13165.0,Furniture,Bookcases
1208,FUR-BO-10000468,O'Sullivan 2-Shelf Heavy-Duty Bookcases,4858.0,4081.0,Furniture,Bookcases
1657,FUR-BO-10000711,"Hon Metal Bookcases, Gray",7098.0,5182.0,Furniture,Bookcases
...,...,...,...,...,...,...
1539,TEC-PH-10004912,Cisco SPA112 2 Port Phone Adapter,5495.0,3901.0,Technology,Phones
1099,TEC-PH-10004922,RCA Visys Integrated PBX 8-Line Router,6698.0,5024.0,Technology,Phones
1192,TEC-PH-10004924,"SKILCRAFT Telephone Shoulder Rest, 2"" x 6.5"" x...",739.0,539.0,Technology,Phones
1174,TEC-PH-10004959,Classic Ivory Antique Telephone ZL1810,10049.0,7537.0,Technology,Phones


In [769]:
Products = Products.drop_duplicates(subset=["ProductID", "ProductName"])

In [770]:
Products[Products.groupby('ProductID')['ProductID'].transform('size') > 2].sort_values(by=['ProductID'])

,ProductID,ProductName,ProductPrice,SupplierPrice,Category,SubCategory


In [771]:
#So we have a bunch of products who share IDs and all do seem to belong in the correct category.
#So we must manage them with as little fuss as possible. If stock numbers were known then we would change ones with lower num. items in stock (less work)
#But we don't know this so let's modify every second one. As we can see above, never more than one duplicate.

In [772]:
#Let's find the biggest value in the column and work from there. 
#I imagine all products are added iteratively i.e. a new product is assigned the next available number regardless of category and subcategory.
Products['ProductID'].str[-4:].max()

'4999'

In [773]:
# Sort the DataFrame based on 'ProductID' to make sure the IDs are in order
Products.sort_values('ProductID', inplace=True)

# Calculate the current_count based on the maximum value of the last four digits
current_count = int(Products['ProductID'].str[-4:].max())

# Iterate through the DataFrame and update the duplicated ProductIDs
for index, row in Products.iterrows():
    product_id = row['ProductID']
    
    # Check if the current ProductID is a duplicate
    if Products[Products['ProductID'] == product_id].shape[0] > 1:
        # Increment the count and update the ProductID
        current_count += 1
        new_product_id = f"{product_id[:-4]}{current_count:04d}"
        Products.loc[index, 'ProductID'] = new_product_id

# Reset the index
Products.reset_index(drop=True, inplace=True)

In [774]:
Products['ProductID'].str[-4:].max()



'5031'

In [775]:
#Now I want to output CSVs just for flexibility and to be safe.
list_of_tables = ['Customers', 'Address', 'Products', 'Shipment', 'OrderDetails']
dataframes = [Customers, Address, Products, Shipment, OrderDetails]

In [776]:
#Here we take the list of tables and dataframes we want to turn into .csvs and save them to the directory we defined in the beginning
for i in range (len(list_of_tables)):
    x = dataframes[i]
    y = list_of_tables[i]
    x.to_csv(f"{path}/{y}.csv", index=False)

In [777]:
#For safety I am also saving the full, final dataframe one folder up in the same directory.  
up_path = os.path.dirname(os.path.dirname(path))
superstore.to_csv(f'{up_path}/db.csv', index=False)

In [778]:
#Lazy way to see longest value in each column. Useful later when creating db table constraints.
for col in superstore:
    print(f'Longest val in {col} is {superstore[col].astype(str).map(len).max()}, min is {superstore[col].astype(str).map(len).min()}')

Longest val in RowID is 4, min is 1
Longest val in OrderID is 14, min is 14
Longest val in OrderDate is 10, min is 10
Longest val in ShipDate is 10, min is 10
Longest val in ShipMode is 14, min is 8
Longest val in CustomerID is 8, min is 8
Longest val in CustomerName is 22, min is 7
Longest val in CustomerType is 11, min is 8
Longest val in Country is 13, min is 13
Longest val in City is 17, min is 4
Longest val in State is 20, min is 4
Longest val in PostalCode is 5, min is 4
Longest val in Region is 7, min is 4
Longest val in ProductID is 15, min is 15
Longest val in Category is 15, min is 9
Longest val in SubCategory is 11, min is 3
Longest val in ProductName is 127, min is 5
Longest val in Sales is 9, min is 4
Longest val in Quantity is 2, min is 1
Longest val in Discount is 4, min is 3
Longest val in Profit is 9, min is 3
Longest val in ProductPrice is 8, min is 4
Longest val in SupplierPrice is 8, min is 4
Longest val in SalesID is 11, min is 11


In [779]:
OrderDetails.columns


Index(['SalesID', 'OrderID', 'ProductID', 'Quantity', 'Discount', 'OrderDate'], dtype='object')